# Random uncertainty data

This notebook create creates some random uncertainty for data records in the V-FOR-WaTer database to demonstrate the uncertainty analysis capabilities.

In [35]:
import pandas as pd
from metacatalog import api, models

Connect the database

In [3]:
session = api.connect_database()

## User

A new user is created, who is holding all the random data. That makes it easier to remove the random data from the database again.

In [9]:
owner = api.find_person(session, first_name='Alfred, E.', last_name='Neumann', return_iterator=True).first()
if owner is None:
    owner = models.Person(first_name='Alfred, E.', last_name='Neumann')
    session.add(owner)
    session.commit()
else:
    print('Found: %s' % str(owner))

Found: Alfred, E. Neumann <ID=2>


### Find Temperature variable

In [17]:
temperature = api.find_variable(session, name='air temperature', return_iterator=True).one()
print('Using: %s' % str(temperature))

Using: air temperature [C] <ID=1>


### Find Licenses

In [30]:
lic = api.find_license(session, short_title="ODbL", return_iterator=True).one()
print('Using %s' % str(lic))

Using Open Data Commons Open Database License <ID=4>


## Entry

We build an `Entry` for some example data. The data was recorded with two HOBO sensor in 2018/2019 on my south and north balcony.

In [34]:
# add the two entries
south = api.add_entry(session, 'HOBO test Sensor South', author=owner.id, location=(7.8558,48.019456), 
                      variable=temperature.id, external_id='10350083', license=lic.id)
north = api.add_entry(session, 'HOBO test Sensor North', author=owner.id, location=(7.8559,48.019555), 
                      variable=temperature.id, external_id='10350083', license=lic.id)

# add sensor information
api.add_details_to_entries(session, [south, north], sensor_precision=0.5)

## Data

Upload the data

In [62]:
south_data = pd.read_csv('example2.csv', usecols=[0,2], index_col=['tstamp'])
north_data = pd.read_csv('example.csv', index_col=['tstamp'])
north_data.columns = ['value', 'precision']
south_data.columns = ['value']

In [65]:
# Create Data sources
if north.datasource is None:
    north.create_datasource('timeseries', 1, 'timeseries', commit=True)
if south.datasource is None:
    south.create_datasource('timeseries', 1, 'timeseries', commit=True)

In [ ]:
# Import data
south.import_data(south_data)

In [73]:
north.import_data(north_data)